In [ ]:
!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper

Obtaining bq_helper from git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
  Updating ./src/bq-helper clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 8615a7f6c1663e7f2d48aa2b32c2dbcb600a440f
  Found existing installation: bq-helper 0.4.1
    Can't uninstall 'bq-helper'. No files were found to uninstall.
  Running setup.py develop for bq-helper


In [ ]:
from bq_helper import BigQueryHelper
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
project_id = "project297814"
%env GCLOUD_PROJECT=project_id

env: GCLOUD_PROJECT=project_id


In [ ]:
from google.cloud import bigquery
import pandas as pd
client=bigquery.Client(project=project_id)

In [ ]:
bq_assistant = BigQueryHelper("bigquery-public-data", "github_repos")
bq_assistant.client=bigquery.Client(project=project_id)

In [ ]:
QUERY1 = '''
 WITH filtered_snp_calls AS (
   SELECT
     reference_name,
     c.name,
     CONCAT(reference_bases, '->', alternate_bases[ORDINAL(1)].alt) AS mutation
   FROM
     `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` AS v, UNNEST(v.call) AS c
   WHERE
     # Only include biallelic SNPs.
     reference_bases IN ('A','C','G','T')
     AND alternate_bases[ORDINAL(1)].alt IN ('A','C','G','T')
     AND (ARRAY_LENGTH(alternate_bases) = 1
       OR (ARRAY_LENGTH(alternate_bases) = 2 AND alternate_bases[ORDINAL(2)].alt = '<*>'))
     # Skip homozygous reference calls and no-calls.
     AND EXISTS (SELECT g FROM UNNEST(c.genotype) AS g WHERE g > 0)
     AND NOT EXISTS (SELECT g FROM UNNEST(c.genotype) AS g WHERE g < 0)
     # Include only high quality calls.
     AND NOT EXISTS (SELECT ft FROM UNNEST(c.filter) ft WHERE ft NOT IN ('PASS', '.'))
 ),

 mutation_type_counts AS (
   SELECT
     reference_name,
     name,
     SUM(CAST(mutation IN ('A->G', 'G->A', 'C->T', 'T->C') AS INT64)) AS transitions,
     SUM(CAST(mutation IN ('A->C', 'C->A', 'G->T', 'T->G',
                           'A->T', 'T->A', 'C->G', 'G->C') AS INT64)) AS transversions
   FROM filtered_snp_calls
   GROUP BY
     reference_name,
     name
 )

 SELECT
   reference_name,
   name,
   transitions,
   transversions,
   transitions/transversions AS titv
 FROM mutation_type_counts
 WHERE
   transversions > 0
 ORDER BY
   titv DESC,
   name
'''

In [ ]:
bq_assistant.estimate_query_size(QUERY1)

5.950535892508924

One

In [ ]:
Query2='''
#standardSQL
 SELECT
   COUNT(1) AS number_of_rows
 FROM
   `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`
'''

In [ ]:
bq_assistant.estimate_query_size(Query2)

0.0

In [ ]:
df=bq_assistant.query_to_pandas(Query2)

In [ ]:
df

,number_of_rows
0,105923159


Two

In [ ]:
Query3='''
#standardSQL
SELECT
  SUM(ARRAY_LENGTH(call)) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`
'''

In [ ]:
bq_assistant.estimate_query_size(Query3)

0.0

In [ ]:
df=bq_assistant.query_to_pandas(Query3)

In [ ]:
df

,number_of_calls
0,182104652


Three

In [ ]:
Query4='''
#standardSQL
SELECT
  COUNT(call) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
'''

In [ ]:
bq_assistant.estimate_query_size(Query4)

0.0

In [ ]:
df=bq_assistant.query_to_pandas(Query4)

In [ ]:
df

,number_of_calls
0,182104652


To count the number of variant segments in the table, first run a query to filter out the non-variant segments:

In [ ]:
Query5 = '''
#standardSQL
SELECT
  COUNT(1) AS number_of_real_variants
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.alternate_bases) AS alt
          WHERE
            alt.alt NOT IN ("<NON_REF>", "<*>"))
'''

In [ ]:
bq_assistant.estimate_query_size(Query5)

0.5340448450297117

In [ ]:
df = bq_assistant.query_to_pandas(Query5)

In [ ]:
df

,number_of_real_variants
0,38549388


To count the number of non-variant segments in the table, first run a query to filter out the non-variant segments:

In [ ]:
Query6 = '''
#standardSQL
SELECT
  COUNT(1) AS number_of_non_variants
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
WHERE
  NOT EXISTS (SELECT 1
                FROM UNNEST(v.alternate_bases) AS alt
              WHERE
                alt.alt NOT IN ("<NON_REF>", "<*>"))
'''

In [ ]:
bq_assistant.estimate_query_size(Query6)

0.5340448450297117

In [ ]:
df = bq_assistant.query_to_pandas(Query6)

In [ ]:
df

,number_of_non_variants
0,143555264


To count the number of rows in which each call set appears

In [ ]:
Query7 = '''
#standardSQL
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
GROUP BY
  call_name
ORDER BY
  call_name
'''

In [ ]:
bq_assistant.estimate_query_size(Query7)

1.5263835601508617

In [ ]:
df = bq_assistant.query_to_pandas(Query7)

In [ ]:
df

,call_name,call_count_for_call_set
0,NA12877,31592135
1,NA12878,28012646
2,NA12889,31028550
3,NA12890,30636087
4,NA12891,33487348
5,NA12892,27347886


Filter out the non-variant segments to count just the variant rows:

In [ ]:
Query8 = '''
#standardSQL
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.alternate_bases) AS alt
          WHERE
            alt.alt NOT IN ("<NON_REF>", "<*>"))
GROUP BY
  call_name
ORDER BY
  call_name
'''

In [ ]:
bq_assistant.estimate_query_size(Query8)

2.0604284051805735

In [ ]:
df = bq_assistant.query_to_pandas(Query8)

In [ ]:
df

,call_name,call_count_for_call_set
0,NA12877,6284275
1,NA12878,6397315
2,NA12889,6407532
3,NA12890,6448600
4,NA12891,6516669
5,NA12892,6494997


To filter the variants by genotype

In [ ]:
Query9 = '''
#standardSQL
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt > 0)
  AND NOT EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt < 0)
GROUP BY
  call_name
ORDER BY
  call_name
'''

In [ ]:
bq_assistant.estimate_query_size(Query9)

4.239954333752394

In [ ]:
df=bq_assistant.query_to_pandas(Query9)

In [ ]:
df

,call_name,call_count_for_call_set
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


Alternatively Efficient

In [ ]:
Query10 = '''
#standardSQL
SELECT
  COUNT(DISTINCT call.name) AS number_of_callsets
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v,  v.call
'''

In [ ]:
bq_assistant.estimate_query_size(Query10)

1.5263835601508617

In [ ]:
df=bq_assistant.query_to_pandas(Query10)

In [ ]:
df

,number_of_callsets
0,6


To count the number of variants per chromosome, you can run the following query, which:

In [ ]:
Query11 = '''
#standardSQL
SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  reference_name
ORDER BY
  CASE
    WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
      THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
      ELSE REGEXP_REPLACE(reference_name, '^chr', '')
  END
'''

In [ ]:
bq_assistant.estimate_query_size(Query11)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Query11)

In [ ]:
df

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr2,646401
2,chr3,542315
3,chr4,578600
4,chr5,496202
5,chr6,512152
6,chr7,459506
7,chr8,416376
8,chr9,344985
9,chr10,396773


The following query shows how to view the per-variant-call FILTER values for the dataset:



In [ ]:
Query12 = '''
#standardSQL
SELECT
  call_filter,
  COUNT(call_filter) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v,
  v.call,
  UNNEST(call.FILTER) AS call_filter
GROUP BY
  call_filter
ORDER BY
  number_of_calls
'''

In [ ]:
bq_assistant.estimate_query_size(Query12)

0.24804932065308094

In [ ]:
df = bq_assistant.query_to_pandas(Query12)

In [ ]:
df

,call_filter,number_of_calls
0,RefCall,11681534
1,PASS,26867854


When analyzing variants, you might want to filter out lower quality variants. If the FILTER column contains the value PASS, it is expected that the column will contain no other values. You can verify this by running the following query. The query also omits any calls that do not contain a PASS value under FILTER.

In [ ]:
Query13 = '''
#standardSQL
SELECT
  reference_name,
  start_position,
  end_position,
  reference_bases,
  call.name AS call_name,
  (SELECT STRING_AGG(call_filter) FROM UNNEST(call.FILTER) AS call_filter) AS filters,
  ARRAY_LENGTH(call.FILTER) AS filter_count
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter = 'PASS')
  AND ARRAY_LENGTH(call.FILTER) > 1
ORDER BY
  filter_count DESC, reference_name, start_position, end_position, reference_bases, call_name
LIMIT
  10
'''

In [ ]:
bq_assistant.estimate_query_size(Query13)

4.281298340298235

In [ ]:
df = bq_assistant.query_to_pandas(Query13)

In [ ]:
df

,reference_name,start_position,end_position,reference_bases,call_name,filters,filter_count


Alternate Five

In [ ]:
Query14 = '''
#standardSQL
SELECT
  call.name AS call_name,
  COUNT(1) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
GROUP BY
  call_name
ORDER BY
  call_name
'''

In [ ]:
bq_assistant.estimate_query_size(Query14)

1.7744328808039427

In [ ]:
df = bq_assistant.query_to_pandas(Query14)

In [ ]:
df

,call_name,number_of_calls
0,NA12877,29795946
1,NA12878,26118774
2,NA12889,29044992
3,NA12890,28717437
4,NA12891,31395995
5,NA12892,25349974


Alternate Six

In [ ]:
QUERY15 = '''
#standardSQL
SELECT
  call.name AS call_name,
  COUNT(1) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
  AND EXISTS (SELECT 1 FROM UNNEST(call.genotype) as gt WHERE gt > 0)
GROUP BY
  call_name
ORDER BY
  call_name
'''

In [ ]:
bq_assistant.estimate_query_size(QUERY15)

4.488003654405475

In [ ]:
df = bq_assistant.query_to_pandas(QUERY15)

In [ ]:
df

,call_name,number_of_calls
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


Best Practice

In [ ]:
Q1 = '''
#standardSQL
SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call
          WHERE EXISTS (SELECT 1
                          FROM UNNEST(call.genotype) AS gt
                        WHERE gt > 0))
GROUP BY
  reference_name
ORDER BY
  reference_name
'''

In [ ]:
bq_assistant.estimate_query_size(Q1)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q1)

In [ ]:
df.head()

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr10,396773
2,chr11,391260
3,chr12,382841
4,chr13,298044


Changing Exist to Join in call.genotype

In [ ]:
Q2 = '''
#standardSQL
SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  reference_name
ORDER BY
  reference_name
'''

In [ ]:
bq_assistant.estimate_query_size(Q2)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q2)

In [ ]:
df.head()

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr10,396773
2,chr11,391260
3,chr12,382841
4,chr13,298044


To sort the output numerically, first remove the "chr" prefix from the reference_name column and give it the alias chromosome:

In [ ]:
Q3 = '''
#standardSQL
SELECT
  REGEXP_REPLACE(reference_name, '^chr', '') AS chromosome,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  chromosome
ORDER BY
  chromosome
'''

In [ ]:
bq_assistant.estimate_query_size(Q3)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q3)

In [ ]:
df.head()

,chromosome,number_of_variant_rows
0,1,615000
1,10,396773
2,11,391260
3,12,382841
4,13,298044


To instead sort the output numerically, cast the chromosome column from a string to an integer:

In [ ]:
Q4 = '''
#standardSQL
SELECT
  CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) AS chromosome,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  chromosome
ORDER BY
  chromosome
'''

In [ ]:
bq_assistant.estimate_query_size(Q4)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q4)

BadRequest: ignored

In this case, the query returns an error because not all chromosome names, such as "X," "Y," and "M" are numeric. Instead, use the CASE function to prepend a "0" to chromosomes 1 through 9 and remove the "chr" prefix:

In [ ]:
Q5 = '''
#standardSQL
SELECT
  CASE
    WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
      THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
      ELSE REGEXP_REPLACE(reference_name, '^chr', '')
  END AS chromosome,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  chromosome
ORDER BY
  chromosome
'''

In [ ]:
bq_assistant.estimate_query_size(Q5)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q5)

In [ ]:
df

,chromosome,number_of_variant_rows
0,01,615000
1,02,646401
2,03,542315
3,04,578600
4,05,496202
5,06,512152
6,07,459506
7,08,416376
8,09,344985
9,10,396773


As a last improvement on the output, display the reference_name column again instead of setting it to the chromosome alias. To do so, move the CASE clause to the ORDER BY function:

In [ ]:
Q6 = '''
#standardSQL
SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  reference_name
ORDER BY
  CASE
    WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
      THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
      ELSE REGEXP_REPLACE(reference_name, '^chr', '')
  END
'''

In [ ]:
bq_assistant.estimate_query_size(Q6)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q6)

In [ ]:
df.head()

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr2,646401
2,chr3,542315
3,chr4,578600
4,chr5,496202


**User Defined Functions**


BigQuery supports User-defined functions, which enable you to create a function using another SQL expression or another programming language, such as JavaScript.

The example in Condensing queries shows how to build a complex query, but the query becomes overly verbose.

The following query demonstrates how to make the query more concise by moving the CASE logic into a function:

In [ ]:
Q7 = '''
#standardSQL
CREATE TEMPORARY FUNCTION SortableChromosome(reference_name STRING)
  RETURNS STRING AS (
  -- Remove the leading "chr" (if any) in the reference_name
  -- If the chromosome is 1 - 9, prepend a "0" since
  -- "2" sorts after "10", but "02" sorts before "10".
  CASE
    WHEN SAFE_CAST(REGEXP_REPLACE(reference_name, '^chr', '') AS INT64) < 10
      THEN CONCAT('0', REGEXP_REPLACE(reference_name, '^chr', ''))
      ELSE REGEXP_REPLACE(reference_name, '^chr', '')
  END
);

SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  reference_name
ORDER BY SortableChromosome(reference_name)
'''

In [ ]:
bq_assistant.estimate_query_size(Q7)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q7)

In [ ]:
df.head()

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr2,646401
2,chr3,542315
3,chr4,578600
4,chr5,496202


Reusing code makes it more concise

In [ ]:
Q8 = '''
#standardSQL
CREATE TEMPORARY FUNCTION SortableChromosome(reference_name STRING)
  RETURNS STRING LANGUAGE js AS """
  // Remove the leading "chr" (if any) in the reference_name
  var chr = reference_name.replace(/^chr/, '');

  // If the chromosome is 1 - 9, prepend a "0" since
  // "2" sorts after "10", but "02" sorts before "10".
  if (chr.length == 1 && '123456789'.indexOf(chr) >= 0) {
    return '0' + chr;
  }

  return chr;
""";

SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  reference_name
ORDER BY SortableChromosome(reference_name)
'''

In [ ]:
bq_assistant.estimate_query_size(Q8)

3.341447635553777

In [ ]:
df = bq_assistant.query_to_pandas(Q8)

In [ ]:
df.head()

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr2,646401
2,chr3,542315
3,chr4,578600
4,chr5,496202
